In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN

from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
from sklearn.model_selection import cross_val_predict, train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, silhouette_score

from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import Birch

In [70]:
clusters = pd.read_csv('labelled_data.csv')
clusters.drop(['Unnamed: 0'], axis = 1, inplace = True)
data = pd.read_csv('data.csv')
data_fil = pd.read_csv('data_.csv')
data_fil.drop(['Unnamed: 0'], axis = 1, inplace = True)
components = pd.read_csv('components.csv')
components.drop(['Unnamed: 0'], axis = 1, inplace = True)
pca = pd.read_csv('X_pca.csv')
pca.drop(['Unnamed: 0'], axis = 1, inplace = True)
features = pd.read_csv('features.csv')
features.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [24]:
data = pd.read_csv('data.csv')
data_raw = data.drop(['Unnamed: 0', ' Name', 'Name', 'Game', 'GameId', 'LudiiRuleset', 'MainRuleset', 'RulesetId'], axis = 1)
data_raw

,Discrete_AB,Realtime_AB,Stochastic_AB,AsymmetricPiecesType_AB,Solitaire_AB,TwoPlayer_AB,Multiplayer_AB,Team_AB,DeductionPuzzle_AB,PlanningPuzzle_AB,...,Negation,Union,Intersection,Complement,ConditionalStatement,ControlFlowStatement,CopyContext,DoLudeme,PlayoutsPerSecond,MovesPerSecond
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17027.06,526666.35
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,8357.74,390094.85
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,952.89,58126.17
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,594.84,166554.78
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,856.62,16799.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2558.16,155799.57
1100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22873.43,466904.64
1101,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,59.72,8100.79
1102,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,359.27,103774.21


In [5]:
components

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-1.110223e-16,6.938894e-17,-6.418477e-17,5.204170e-17,6.505213e-19,1.994932e-17,-4.987330e-17,-1.084202e-18,8.971773e-18,-3.133344e-17,...,-5.087711e-18,4.138359e-18,-4.732670e-17,-7.905152e-17,2.994392e-18,-2.951655e-17,-5.905717e-17,-2.435985e-17,5.400259e-18,7.859704e-18
1,0.000000e+00,-0.000000e+00,-3.053113e-16,2.220446e-16,1.283695e-16,6.938894e-17,-2.116363e-16,1.630640e-16,-5.637851e-17,-8.673617e-19,...,1.410171e-17,-1.491861e-17,3.082313e-17,2.822413e-17,5.229072e-17,-3.806974e-17,1.186638e-16,1.669304e-17,-5.286555e-17,2.520720e-17
2,-3.781949e-02,9.413771e-02,-1.815347e-02,1.262619e-01,1.183378e-02,-4.102541e-03,3.017488e-02,-2.632164e-02,2.943474e-03,-2.640667e-02,...,2.863916e-02,-3.367406e-02,-1.079407e-02,-3.264122e-02,5.293016e-02,-2.190786e-02,-1.781398e-02,-4.647145e-02,-2.516127e-04,-2.408027e-02
3,1.536356e-02,-2.748959e-02,-3.703336e-02,7.018973e-03,2.948472e-02,3.601250e-02,-2.472529e-02,2.370673e-02,3.332242e-02,-7.620208e-03,...,1.372046e-01,4.503487e-02,-2.766354e-02,-2.356731e-02,-9.462058e-02,-3.784135e-02,-6.556932e-02,4.537029e-02,6.565066e-02,2.909197e-02
4,4.384985e-04,-3.046630e-02,4.616578e-02,6.870980e-03,4.890332e-02,8.198423e-02,1.639145e-01,1.269479e-02,-1.449603e-03,-5.649335e-02,...,1.564988e-02,-3.880800e-03,4.212661e-03,-2.125417e-02,-2.222566e-03,-1.081256e-02,-2.297326e-02,1.548598e-02,-1.004979e-02,1.691781e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,4.634502e-02,5.995800e-02,-8.969029e-02,2.259470e-02,5.212248e-02,8.925715e-03,-3.070099e-02,-4.552678e-04,-3.975971e-03,-1.059312e-02,...,-1.747912e-02,1.168089e-02,-3.944577e-02,-1.362644e-02,4.557440e-02,-2.664549e-03,-1.583508e-02,3.495718e-02,-1.680971e-03,-3.446073e-02
658,-4.721864e-02,1.180804e-01,2.702262e-02,-1.989761e-02,-5.424233e-02,-2.237628e-02,4.001367e-02,-2.652166e-03,2.353085e-02,3.261074e-02,...,5.691399e-02,-7.858158e-03,8.666936e-03,-1.119899e-02,2.742659e-02,-2.939515e-03,1.079850e-02,-1.129421e-02,-4.681072e-02,3.779300e-02
659,1.527624e-02,1.155072e-01,-2.783306e-03,5.288256e-02,-7.631493e-02,-1.726677e-02,1.319530e-02,-1.472842e-02,3.093784e-02,1.772525e-03,...,6.756337e-02,4.546232e-03,2.729716e-02,-1.274112e-02,3.981689e-02,7.354509e-03,-2.872676e-02,1.692887e-02,-6.393491e-02,3.438223e-02
660,-6.246949e-03,-6.034446e-02,-1.800851e-02,2.162270e-02,-1.436175e-02,3.339365e-02,3.066937e-02,4.185401e-02,-1.076291e-01,-7.721158e-02,...,3.665129e-02,-1.246483e-02,-1.073381e-03,1.802502e-02,1.015354e-02,-5.524225e-03,3.145427e-02,-1.726443e-02,-2.367771e-02,1.659663e-02


In [6]:
pca

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-2.377,-6.673,-5.851,-0.561,2.331,4.363,-1.378,2.452,-0.370,-0.083,...,0.379,-0.644,-0.416,-0.843,-0.395,0.870,-0.068,-0.397,0.605,-0.605
1,-0.006,-5.941,-5.855,2.475,0.225,3.474,-3.962,-0.798,-2.371,0.101,...,0.261,-0.871,1.108,0.163,0.767,0.208,-0.137,0.003,1.112,-0.128
2,1.036,-6.752,4.593,1.441,-5.910,-4.895,0.351,-3.561,4.036,-1.800,...,0.327,-1.130,-0.819,-0.457,-0.382,-0.289,-0.360,1.055,-0.016,-1.910
3,4.982,0.437,-1.177,-3.169,19.634,-10.483,4.229,7.963,0.458,5.838,...,0.397,-0.118,-1.783,-0.676,-0.710,0.208,-0.359,0.614,0.089,0.312
4,-2.031,-3.990,-0.796,-0.936,-4.539,-3.492,-2.088,-1.940,1.124,1.435,...,-0.578,1.275,0.006,-0.563,0.745,-0.964,-0.305,0.623,1.886,0.776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,0.639,-2.976,-2.050,1.517,-1.798,1.197,-2.003,-0.571,-2.601,1.747,...,-1.098,-1.870,1.056,0.515,0.679,-1.922,-0.314,0.628,-1.112,-1.028
1100,-1.709,-5.271,-2.818,4.563,-1.080,4.678,-0.577,1.306,-5.732,0.534,...,-0.217,0.577,0.160,-0.057,-1.163,0.024,0.801,-0.124,0.444,-0.427
1101,10.832,6.121,-3.783,-5.151,-6.788,1.455,1.763,-0.712,-2.876,-0.250,...,1.317,0.224,-1.949,0.434,0.207,1.047,0.039,0.197,0.231,0.527
1102,3.862,-4.336,-4.047,-1.978,4.875,0.209,-2.565,-1.370,3.393,0.041,...,-0.257,0.154,0.597,-0.102,0.308,0.674,0.190,0.216,0.484,0.645


In [7]:
features['0'] = features['0'].apply(str)
features['0']

0     38
1     31
2      9
3     26
4     20
      ..
70     2
71    70
72    41
73    86
74    36
Name: 0, Length: 75, dtype: object

In [41]:
exp = components[features['0']]
exp

,38,31,9,26,20,52,29,24,71,45,...,64,106,43,65,124,2,70,41,86,36
0,1.633969e-17,-9.293645e-17,-3.133344e-17,1.333569e-17,3.056095e-17,7.182940e-17,-2.965632e-17,-4.524172e-17,1.179104e-16,-5.161162e-17,...,-3.494445e-17,-9.128325e-18,-2.725667e-17,1.735115e-17,6.055847e-17,-6.418477e-17,-4.063244e-17,-5.188183e-18,8.300600e-17,1.710837e-17
1,-3.787931e-17,-3.306817e-17,-8.673617e-19,-6.158268e-17,3.100818e-17,2.558717e-17,-6.006480e-17,5.187907e-17,2.969359e-17,2.022715e-17,...,2.055664e-17,1.348011e-17,1.905485e-17,3.337310e-18,2.001475e-17,-3.053113e-16,-5.818269e-18,3.686965e-17,1.096908e-19,-5.631075e-17
2,-2.988841e-03,-2.140427e-02,-2.640667e-02,8.407326e-04,1.126600e-02,2.563319e-02,-3.013622e-02,3.607456e-02,-3.478031e-03,6.942575e-05,...,-3.244061e-02,-3.694395e-02,4.053656e-03,1.221100e-03,2.658300e-02,-1.815347e-02,4.728045e-03,-7.504843e-04,9.537092e-03,9.305675e-03
3,5.310651e-02,-3.947043e-02,-7.620208e-03,1.355750e-03,-7.682362e-04,-4.554822e-02,-9.602895e-03,2.226478e-02,2.228324e-02,-2.921820e-02,...,-1.746864e-02,2.589572e-02,-5.195780e-03,1.924565e-02,4.467020e-02,-3.703336e-02,3.800504e-02,3.321259e-02,6.646318e-02,2.313922e-02
4,-2.108666e-03,6.288320e-03,-5.649335e-02,5.234297e-02,3.348187e-02,-2.214482e-02,-3.897516e-03,1.660093e-03,1.800890e-02,2.856835e-02,...,8.301722e-03,2.068195e-03,-2.568263e-02,-1.781369e-02,-1.774063e-02,4.616578e-02,-3.345147e-02,5.953309e-03,1.640924e-02,5.855138e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,-1.373623e-03,-2.444107e-02,-1.059312e-02,4.464017e-03,-4.364204e-02,6.134077e-03,1.663981e-02,-1.054748e-02,-1.211801e-02,-5.779076e-03,...,-1.342660e-02,-3.547178e-02,1.073733e-02,-3.414021e-02,3.789390e-02,-8.969029e-02,1.844899e-02,1.473318e-02,3.394765e-02,4.073428e-03
658,-2.792449e-02,5.671244e-02,3.261074e-02,-7.170814e-03,1.709297e-02,-3.333091e-03,-4.167623e-02,1.053121e-03,7.457332e-03,-3.512675e-03,...,-1.189911e-02,2.408006e-02,1.115878e-02,5.811319e-03,6.775920e-02,2.702262e-02,1.681043e-02,4.233714e-02,-3.927636e-02,1.778864e-02
659,-1.122226e-02,4.018707e-02,1.772525e-03,-2.198962e-02,1.647125e-02,5.791430e-03,-1.974307e-02,-4.787449e-04,4.181020e-03,-1.811307e-02,...,9.336477e-03,1.029693e-02,-5.586680e-03,-2.735168e-02,1.173697e-01,-2.783306e-03,1.326779e-02,3.592927e-02,-2.498468e-02,1.315590e-02
660,4.715402e-02,3.617440e-02,-7.721158e-02,-4.694966e-02,3.119324e-02,-1.671289e-02,-3.731763e-02,3.312544e-02,-6.157383e-03,8.437140e-03,...,3.414664e-02,-3.527659e-02,-8.353870e-03,-6.127731e-03,-4.710333e-03,-1.800851e-02,-3.478203e-02,-2.784070e-02,-2.936225e-03,6.512157e-02


In [66]:
# keep only highest range columns 
centre1 = [[-2.71946866e-01, -6.61476758e-01, -2.50120279e-01, -7.98814071e-02
, -8.54357658e-02, 9.91293979e-01, 3.89520061e-01, 1.38021049e-01
, 0.00000000e+00, -4.26014323e-02, -6.03022689e-02, -1.96495462e-01
, -2.39806440e-01, 8.12133537e-02, -2.77620552e-01, -1.00319709e-01
, -6.74506526e-02, -1.55302088e-01, -3.10363532e-02, -2.06138104e-01
, -2.13200716e-01, -3.63741244e-01, -7.98814071e-02, -2.46005553e-01
, -3.01101032e-02, -6.03022689e-02, -6.41755519e-02, 1.66831700e+00
, -7.98814071e-02, 2.39833460e-01, -6.74506526e-02, 1.28945722e-01
, -5.66373174e-01, 3.32495958e-01, -1.36305138e-01, 8.66168463e-01
, 4.80959703e-01, 5.01949953e-01, -8.13513658e-01, -8.13542514e-01
, -5.11492055e-01, -6.61452746e-01, -6.05028132e-02, 5.78986662e-01
, -2.31612144e-01, -3.47734071e-02, -1.58944358e-01, -2.12305839e-01
, -1.08931886e-01, 4.96605338e-01, 5.01035378e-01, -1.90673942e-01
, -1.87747029e-01, 2.18744712e-01, 3.21606078e-01, 7.16390691e-01
, -4.27020751e-01, 2.90136790e-01, 1.17789992e+00, -1.08755670e-01
, 1.67131568e-01, 4.33652677e-02, -3.01101032e-02, -4.07169123e-01
, 2.45493540e+00, -6.03022689e-02, -1.13331534e-01, 8.52889336e-02
, -9.36249324e-02, 2.27535949e+00, -3.01101032e-02, 1.94494991e-01
, 1.39279849e+00, 5.20632343e-02, -4.05922321e-01, -8.08163236e-02
, 2.82589300e-01, -3.01101032e-02, -7.26361774e-01, -7.19006087e-01
, -7.47217865e-02, -9.58236789e-02, -3.22376560e-01, -3.23000377e-01
, -9.00377596e-02, -4.15035573e-02, -1.98706983e-01, -1.96116772e-01
, -1.47164733e-01, -2.39367344e-01, -2.27166803e-01, -1.96098658e-01
, -1.41391916e-01, -3.01101032e-02, -4.00687211e-02, -3.76206411e-02
, -9.19368705e-02, 1.73877926e+00, 6.76406143e-03, 1.28962821e+00
, 1.06154688e+00, 8.27522800e-01, 1.94138097e-01, -4.26014323e-02
, -3.01101032e-02, 2.27725917e+00, -3.01101032e-02, -1.28639996e-01
, -4.61391407e-01, -3.98367445e-01, -1.30425775e-01, -1.52923293e-01
, -1.91362792e-01, -4.76095229e-01, -2.56197318e-01, -4.47213595e-01
, -6.90606046e-02, -1.57455411e-01, -4.92889889e-02, -5.20971996e-02
, 8.24003698e-01, 8.39947884e-01, -4.71377635e-02, -4.25801680e-02
, -3.01101032e-02, -1.78469778e-02, 1.45912345e+00, -8.97867240e-02
, 5.15990711e-01, 6.01190816e-02, -1.48380793e-01, -1.99480922e-01
, -3.94244975e-02, -5.94264370e-02, -9.33480037e-02, -3.18279621e-02
, -3.01101032e-02, -1.02698696e-01, -3.01101032e-02, -3.32619216e-01
, -1.69973409e-01, -2.06138104e-01, 2.08448190e-01, -6.03022689e-02
, -5.21995751e-02, -8.54357658e-02, -9.06596828e-02, -1.52815589e-01
, -2.97544580e-01, -4.91777154e-01, 1.73609163e-01, -3.49215148e-01
, 1.62533199e+00, 6.03378063e-02, -6.61890394e-02, -3.30979967e-01
, -4.64515432e-01, 6.97815546e-01, 5.66809936e-03, -4.39078954e-01
, 2.03909584e-01, -1.92956267e-01, -1.91362792e-01, -2.01315186e-01
, -8.33672015e-01, -5.97072500e-01, -4.16197043e-01, -6.03022689e-02
, -4.26014323e-02, -3.62142984e-01, -2.03738641e-01, -1.80944409e-01
, -7.98814071e-02, -1.93891684e-01, -4.26014323e-02, -3.81088199e-01
, -1.25057484e-01, -7.98814071e-02, -8.54357658e-02, -2.50984656e-01
, -1.45326365e-01, 4.74319169e-01, 1.87354821e+00, -1.99763377e-01
, -1.82384002e-01, -3.98685158e-02, -1.50972441e-01, -1.40572179e-01
, -3.01101032e-02, -8.42343957e-02, -6.74506526e-02, -4.16274081e-02
, -6.74457155e-02, -2.52210917e-01, -2.45883851e-01, -7.77494301e-02
, -7.03780241e-02, 1.68335563e-01, 1.70624096e-01, -3.43431110e-02
, -5.37117430e-02, 1.38663756e+00, 1.39765009e+00, -6.31987365e-02
, -5.26942156e-02, -4.37531668e-02, -1.40467299e-03, 4.37526795e-02
, -3.01101032e-02, -3.01101032e-02, 9.04531357e-03, 1.86638257e-02
, -4.23682186e-02, -3.26114680e-01, -2.82347931e-01, -8.26946646e-02
, -3.01101032e-02, -9.25820969e-02, -3.01101032e-02, -8.79788604e-02
, -5.13091508e-01, -2.25884272e-01, 1.94494991e-01, -3.80969659e-02
, 1.40932093e+00, 8.49617406e-02, -4.06314403e-01, -9.87937360e-02
, 1.67491240e-01, -3.01101032e-02, -7.22797448e-01, -7.12513673e-01
, -6.65135403e-02, -1.39705491e-01, -3.33686559e-01, -3.31606828e-01
, -1.65435255e-01, -3.89834542e-02, -2.14239088e-01, -2.08152244e-01
, -2.23196518e-01, -2.69735358e-01, -2.66269910e-01, -2.53497145e-01
, -1.30270721e-01, -3.01101032e-02, -5.25447938e-02, -5.23236974e-02
, -9.30456710e-02, 1.78932890e+00, 3.88011436e-02, 1.40898503e+00
, 9.97304684e-01, 8.23273626e-01, 1.94494991e-01, -3.01101032e-02
, 2.27877227e+00, -1.35446623e-01, -4.64134803e-01, -3.99071663e-01
, -1.35398107e-01, -1.52877191e-01, -7.78307902e-02, -2.22306039e-01
, -4.94532891e-02, 8.30349583e-01, 8.46404661e-01, -4.95696083e-02
, -4.00833644e-02, 1.12079168e-02, -2.14992765e-01, -2.58028306e-01
, -4.25633848e-02, -7.95347568e-02, -1.20148874e-01, -3.85984364e-02
, -4.54189000e-02, -1.08660924e-01, -2.25483836e-01, -2.10203786e-01
, -3.48706157e-02, -1.51979156e-01, -1.35950551e-01, -3.01101032e-02
, -8.54357658e-02, -3.64157429e-02, -6.74506526e-02, -2.67635841e-01
, -2.61264929e-01, -7.98376846e-02, -7.31924380e-02, 4.83197335e-02
, 5.72211486e-02, -3.09585599e-02, -4.45895797e-02, 1.47955729e+00
, 1.49987939e+00, -7.11169254e-02, -5.90366865e-02, -5.21879947e-02
, -1.76895270e-02, 2.99974676e-02, -3.01101032e-02, 6.97660316e-03
, 1.56057180e-02, -4.23947883e-02, -3.52893702e-01, -3.09299598e-01
, -8.20678278e-02, -8.46768348e-02, -3.01101032e-02, -9.31415748e-02
, -4.41307036e-01, -5.54577450e-01, -1.36092136e-01, 8.66402017e-01
, 4.81180494e-01, -8.14727931e-01, -8.13581133e-01, -5.12924284e-01
, -6.61411449e-01, 5.79327440e-01, -2.31374332e-01, -1.58631681e-01
, -2.12091590e-01, -1.08573203e-01, 4.96789421e-01, 5.01219486e-01
, -4.27867080e-01, -1.08540010e-01, -3.91924758e-01, -3.01101032e-02
, 1.94494991e-01, -3.01101032e-02, 1.32726926e+00, 1.73046348e-01
, -4.09109098e-01, -1.83979445e-01, 1.86307835e-01, -7.23302491e-01
, -7.11063768e-01, -6.05714951e-02, -1.60363235e-01, -3.33916835e-01
, -3.32110507e-01, -1.78657368e-01, -3.42226351e-02, -2.16390392e-01
, -2.12233670e-01, -2.00442207e-01, -3.18419904e-01, -3.14927079e-01
, -3.05022397e-01, -1.33668051e-01, -3.01101032e-02, -5.56811869e-02
, -4.44938029e-02, -9.09022251e-02, 1.87225010e+00, 6.61320344e-02
, 1.42944973e+00, 1.15874484e+00, 9.10687085e-01, 1.93673742e-01
, -3.01101032e-02, 2.27076790e+00, -1.47521579e-01, -4.63022610e-01
, -3.99914275e-01, -1.31033499e-01, -1.51631558e-01, -2.54183891e-01
, -4.48671112e-01, -7.15985641e-02, -2.79701398e-01, -4.95353230e-02
, -5.20997669e-02, 8.02699191e-01, 8.18306921e-01, -4.30143690e-02
, -4.03382982e-02, -1.86696345e-03, -2.67373154e-01, -3.08898220e-01
, -4.24540110e-02, -7.68270314e-02, -1.16665382e-01, -4.15851912e-02
, -5.44767702e-02, -1.07602262e-01, 7.42514008e-02, -4.90356440e-01
, 1.70742667e-01, -3.47580469e-01, 1.61759613e+00, 3.19338147e-02
, -5.98776839e-01, 6.76457169e-01, 5.96652935e-02, -4.37565937e-01
, 1.90739472e-01, -1.95451899e-01, -8.30470947e-01, -5.92831248e-01
, -3.82645326e-01, 4.76249311e-01, -2.52158313e-01, -2.28977917e-01
, -6.74506526e-02, -1.45864991e-01, -1.28742323e-01, -3.01101032e-02
, -7.98814071e-02, -6.03022689e-02, -5.21995751e-02, -4.26014323e-02
, -4.26014323e-02, -1.00319709e-01, -9.56073628e-02, -6.74506526e-02
, -5.21995751e-02, -3.83753500e-02, -3.48917419e-02, -3.01101032e-02
, 9.02518538e-01, 9.96465509e-01, -7.39221271e-02, -3.01101032e-02
, -1.25057484e-01, -9.56073628e-02, -6.79092352e-02, -5.84025569e-02
, -4.26014323e-02, -3.22490310e-01, -2.81387430e-01, -8.54357658e-02
, -5.21995751e-02, -4.26014323e-02, -3.01101032e-02, 1.71479611e-01
, -2.30285496e-01]]

centre2 = [[1.94529264e-01, 7.27031127e-01, -1.09684023e-01, -7.98814071e-02
, 1.20033721e-02, 7.78904260e-02, -6.03022689e-02, -1.28742323e-01
, 0.00000000e+00, 1.51745598e-01, -6.03022689e-02, 7.08682072e-01
, 6.70573032e-01, 8.91909399e-01, -8.52407490e-02, -1.00319709e-01
, -6.74506526e-02, -1.55302088e-01, -2.03738641e-01, -2.06138104e-01
, -2.13200716e-01, -3.63741244e-01, -7.98814071e-02, -2.46005553e-01
, -3.01101032e-02, -6.03022689e-02, -5.24239249e-01, -4.57384344e-01
, -7.98814071e-02, 6.60097444e-01, -6.74506526e-02, -9.90858921e-01
, 6.98489908e-01, -1.67156183e-01, 2.33013839e-01, -2.99383920e-02
, 1.33040733e-01, -1.73280313e-01, 6.27609362e-01, 1.45338108e-01
, 8.02246027e-01, 1.02167235e+00, -6.94130514e-02, 3.07927952e-01
, 2.28619292e-01, -3.47734071e-02, 2.38669215e-01, 2.88495383e-01
, 2.06848408e-01, 3.09531053e-01, 2.85162284e-01, 6.29549071e-01
, 6.27668633e-01, 4.18943703e-02, -1.45075647e-01, -1.33337598e-01
, -4.81622497e-01, -4.29853167e-01, -3.73730185e-01, 2.35492692e-01
, 1.16301471e-01, 8.55902016e-02, 2.44613483e-01, 2.07738342e+00
, -2.61404029e-01, -6.03022689e-02, -3.94720220e-02, 7.98111943e-01
, 8.06574874e-01, -3.15143845e-01, 2.44613483e-01, -3.01101032e-02
, -2.75173671e-01, -4.60217669e-02, -4.05168429e-01, 6.56549141e-01
, -2.27692057e-01, 2.44613483e-01, 4.27313970e-01, 4.08553585e-01
, -7.19046001e-02, 2.20334467e-01, 1.18468485e+00, 1.13018028e+00
, 1.17096410e+00, -2.84457268e-02, 1.60580821e+00, 1.58461580e+00
, 1.19556142e+00, 7.13806192e-02, -9.58748433e-02, -1.68719964e-01
, 8.46928448e-03, -3.01101032e-02, 5.83581945e-02, 9.72411556e-03
, -9.19368705e-02, -3.94715368e-01, -1.66143897e-01, -3.73902188e-01
, -2.15441763e-01, -2.45640515e-01, -3.04545019e-02, -4.26014323e-02
, -3.01101032e-02, -2.24393064e-01, -3.01101032e-02, -7.51885032e-02
, -4.61391407e-01, -3.98367445e-01, -1.30425775e-01, -1.52923293e-01
, -1.91362792e-01, -4.76095229e-01, -2.56197318e-01, -4.47213595e-01
, 1.46097772e-01, -1.13582872e-01, -4.92889889e-02, -5.20971996e-02
, -5.86696926e-01, -5.88817915e-01, 5.25250708e-02, -4.25801680e-02
, -3.01101032e-02, -1.63720210e-01, -1.30366468e-01, -1.39250069e-01
, -7.98814071e-02, 2.17610378e-01, -1.48380793e-01, -1.59944096e-01
, -3.94244975e-02, -5.94264370e-02, -9.33480037e-02, -3.18279621e-02
, -3.01101032e-02, -1.02698696e-01, -3.01101032e-02, -3.54093579e-01
, -1.19946578e-01, -1.64342609e-01, 1.24699595e+00, -6.03022689e-02
, -5.21995751e-02, -8.54357658e-02, 1.24875596e-03, 2.06643246e-02
, 8.44240268e-01, 1.55179652e-01, 3.99830110e-01, 2.41218199e+00
, -2.56007692e-01, -6.77498819e-02, 6.62181247e-01, 2.63346022e+00
, 8.61002277e-01, 1.06949962e+00, 1.74554775e+00, 1.93480024e+00
, 2.83936782e-03, 3.93973002e-01, 1.19647366e+00, 1.40413763e-01
, 7.93720861e-01, 1.54053974e+00, 2.14644398e+00, 7.72467081e-02
, -4.26014323e-02, 2.47738723e+00, 1.52842249e+00, 5.49028492e-02
, 1.28357255e-01, 6.02185907e-01, 3.46092628e-01, 2.17701696e+00
, 9.48844025e-01, -7.98814071e-02, -8.54357658e-02, -4.04211700e-01
, -3.90921167e-01, -8.03554575e-01, -1.95880980e-01, -1.99763377e-01
, -1.82384002e-01, -3.98685158e-02, -9.36018635e-02, -7.84619651e-02
, -3.01101032e-02, -8.42343957e-02, -6.74506526e-02, -4.16274081e-02
, -6.74457155e-02, 2.04895315e+00, 1.99755228e+00, 4.99928457e-01
, 5.71748741e-01, -2.80692802e-02, -4.65022929e-02, 2.79002299e-01
, 4.36352424e-01, -2.42186754e-01, -2.26243314e-01, -6.31987365e-02
, -5.26942156e-02, -4.37531668e-02, 4.53098007e-02, 4.01437672e-02
, -3.01101032e-02, 2.44613483e-01, -2.17053647e-01, -2.10906950e-01
, -4.23682186e-02, -2.21228939e-01, -1.62991809e-01, -8.26946646e-02
, 2.44613483e-01, 4.93665241e-01, -3.01101032e-02, 5.57373185e-01
, -1.88912515e-01, -4.64974356e-02, -3.01101032e-02, -3.80969659e-02
, -2.85940560e-01, -4.72719459e-02, -4.05561111e-01, 8.02597046e-01
, -2.13933902e-01, 2.44613483e-01, 3.54046390e-01, 2.93852092e-01
, -6.39311261e-02, 5.30992449e-01, 1.29346258e+00, 1.19409265e+00
, 1.66033972e+00, -2.62804632e-02, 1.73040959e+00, 1.68043355e+00
, 1.81324113e+00, 2.62911197e-02, -1.33562088e-01, -2.24635097e-01
, 3.07654783e-03, -3.01101032e-02, 1.39221991e-01, 6.18370970e-02
, -9.30456710e-02, -3.81928177e-01, -2.19384186e-01, -4.09874518e-01
, -1.96005963e-01, -2.44920260e-01, -3.01101032e-02, -3.01101032e-02
, -2.24435437e-01, -4.39302225e-02, -4.64134803e-01, -3.99071663e-01
, -1.35398107e-01, -1.52877191e-01, 1.66901746e-01, 4.44164558e-02
, -4.94532891e-02, -5.95663007e-01, -5.97622657e-01, 4.96972653e-02
, -4.28380977e-02, 5.46780115e-01, -2.14992765e-01, -2.07048058e-01
, -4.25633848e-02, -7.95347568e-02, -1.20148874e-01, -3.85984364e-02
, -4.54189000e-02, -1.08660924e-01, -2.25483836e-01, -2.10203786e-01
, -3.48706157e-02, -9.55423307e-02, -7.35243639e-02, -3.01101032e-02
, -8.54357658e-02, -3.64157429e-02, -6.74506526e-02, 2.17345655e+00
, 2.12167721e+00, 5.64407569e-01, 5.94612946e-01, -5.14214430e-02
, -9.87459769e-02, -2.31123133e-02, 3.54710311e-01, -2.61429387e-01
, -2.44252177e-01, -7.11169254e-02, -5.90366865e-02, -5.21879947e-02
, 3.02095936e-02, 5.12245421e-02, 2.44613483e-01, -2.14159152e-01
, -2.08682826e-01, -4.23947883e-02, -2.42388300e-01, -2.07382861e-01
, -8.20678278e-02, 4.80686865e-01, -3.01101032e-02, 7.53306770e-01
, -1.57618563e-01, 6.96945706e-01, 2.33261599e-01, -2.97470641e-02
, 1.33234977e-01, 6.27612723e-01, 1.45462238e-01, 8.02372253e-01
, 1.02176446e+00, 3.08240377e-01, 2.28904113e-01, 2.39093375e-01
, 2.88777155e-01, 2.07332830e-01, 3.09716955e-01, 2.85349777e-01
, -4.82774417e-01, 2.35740258e-01, 2.16229562e+00, 2.44613483e-01
, -3.01101032e-02, -3.01101032e-02, -2.71362468e-01, -4.09904921e-02
, -4.08610475e-01, 4.96543139e-01, -1.90653569e-01, 2.76536775e-01
, 1.97162561e-01, -5.55012903e-02, 6.30616013e-01, 1.41138240e+00
, 1.30662646e+00, 1.67231262e+00, -3.42226351e-02, 1.74421864e+00
, 1.70946493e+00, 1.62838587e+00, -6.93160513e-03, -1.87111868e-01
, -2.69640412e-01, -3.37446915e-04, -3.01101032e-02, 1.62047374e-01
, 8.92198483e-03, -9.09022251e-02, -3.48732174e-01, -2.35270357e-01
, -3.84307178e-01, -2.05933381e-01, -2.56282373e-01, -2.72836222e-02
, -3.01101032e-02, -2.24384253e-01, -4.59572296e-02, -4.63022610e-01
, -3.99914275e-01, -1.31033499e-01, -1.51631558e-01, -2.54183891e-01
, -4.48671112e-01, 2.52419850e-01, 6.44619781e-02, -4.95353230e-02
, -5.20997669e-02, -6.03965255e-01, -6.06046815e-01, 4.46333101e-02
, -4.29425954e-02, 6.75189357e-01, -2.67373154e-01, -2.55929503e-01
, -4.24540110e-02, -7.68270314e-02, -1.16665382e-01, -4.15851912e-02
, -5.44767702e-02, -1.07602262e-01, 1.27185075e+00, 1.78652300e-01
, 3.96445307e-01, 2.42398694e+00, -2.33436323e-01, -7.31389922e-02
, 8.85167673e-01, 1.03341160e+00, 1.63837937e+00, 1.94054358e+00
, 3.47430385e-02, 3.86294269e-01, 7.81105589e-01, 1.53289503e+00
, 2.16769553e+00, -8.01907461e-01, -2.52158313e-01, -2.28977917e-01
, -6.74506526e-02, -8.80011932e-02, -6.34845062e-02, -3.01101032e-02
, -7.98814071e-02, -6.03022689e-02, -5.21995751e-02, -4.26014323e-02
, -4.26014323e-02, 8.14993996e-01, 7.76711055e-01, 3.01799862e-01
, 4.24067622e-01, -1.58447969e-01, -1.89914886e-01, 2.44613483e-01
, -1.86216711e-01, -1.67131568e-01, -7.39221271e-02, 2.44613483e-01
, 7.62990486e-02, 1.66088163e-01, -2.13493951e-01, -2.06616106e-01
, -4.26014323e-02, -2.65905869e-01, -2.17995588e-01, -8.54357658e-02
, -5.21995751e-02, -4.26014323e-02, -3.01101032e-02, 4.41480433e-01
, -1.92048761e-02]]

centre3 = [[2.45467774e-02, 2.96836623e-01, -1.93852773e-01, 3.55196418e-03
, -8.54357658e-02, -1.25130225e-01, -6.03022689e-02, 1.06574770e-01
, 0.00000000e+00, -4.26014323e-02, -5.19158607e-03, 9.84332803e-02
, 9.70435185e-02, -1.58725589e-01, -2.64773998e-01, -1.00319709e-01
, 8.04940901e-02, 4.12424749e-01, 3.88710267e-01, 5.47426355e-01
, -2.13200716e-01, -3.63741244e-01, -7.98814071e-02, -2.46005553e-01
, -3.01101032e-02, -6.03022689e-02, -5.08134843e-01, -4.57384344e-01
, -7.98814071e-02, -6.52126836e-01, -6.74506526e-02, 3.64004889e-01
, -1.09867644e-01, -5.93782769e-02, -2.52340036e-01, -3.48848222e-01
, -1.72765233e-01, -2.76298032e-03, -4.83086614e-01, 7.21369825e-02
, -7.39353022e-01, -1.64284974e-01, -6.94130514e-02, -4.63023043e-01
, -2.14951326e-01, 1.28960980e-02, -2.57769872e-01, -2.43119858e-01
, -2.71463933e-01, -5.64000060e-01, -5.70204729e-01, -1.23525717e-01
, -1.25945568e-01, -9.84412136e-02, -6.22116406e-03, -2.56387694e-02
, 3.77152854e-01, 2.62196192e-01, -2.76733130e-02, -2.55817424e-01
, 1.46765867e-01, -2.99457930e-02, -3.01101032e-02, -1.03788881e-01
, -3.94609301e-01, -6.03022689e-02, -1.13331534e-01, -1.29567807e-01
, -7.84515687e-02, -3.56927898e-01, -3.01101032e-02, -3.01101032e-02
, -2.54959197e-01, -4.60217669e-02, -3.79341375e-01, -8.08163236e-02
, 2.10512982e-01, -3.01101032e-02, 1.11355123e+00, 1.13040784e+00
, 9.03810126e-04, -3.49642009e-02, -3.37498416e-01, -3.35386602e-01
, -1.38759831e-01, -4.15035573e-02, -1.98706983e-01, -1.96116772e-01
, -1.47164733e-01, -4.64115761e-02, -4.58592729e-02, -4.41695692e-02
, -2.13044639e-02, -3.01101032e-02, -4.00687211e-02, -3.76206411e-02
, -7.52296503e-02, -2.71055309e-01, 3.88714464e-01, 9.36873441e-02
, -1.76714835e-01, -2.25146570e-01, -3.04545019e-02, -4.26014323e-02
, -3.01101032e-02, -3.70940816e-01, -3.01101032e-02, -1.78507161e-01
, -4.61391407e-01, -3.98367445e-01, -1.30425775e-01, -1.52923293e-01
, -1.91362792e-01, -4.76095229e-01, -2.56197318e-01, -4.47213595e-01
, -4.64828847e-02, -2.15804248e-01, -4.92889889e-02, -5.20971996e-02
, -4.75610249e-01, -4.83059258e-01, -2.39126856e-02, -4.25801680e-02
, -3.01101032e-02, -1.30026856e-01, -2.39731651e-01, -9.07694395e-02
, -7.98814071e-02, -1.35930742e-01, -1.94125754e-02, -4.81728837e-02
, 1.04696844e-01, -5.94264370e-02, 9.17344003e-02, 7.80593066e-02
, -3.01101032e-02, -9.28560054e-02, -3.01101032e-02, 5.29910679e-01
, -1.69973409e-01, -1.72646350e-01, -5.14353589e-01, -6.03022689e-02
, 7.50152834e-02, -8.54357658e-02, 1.30285769e-01, -5.85617145e-02
, -2.29697158e-01, 6.00917903e-02, 7.73454427e-03, -2.96023437e-01
, -2.21143668e-01, 4.22371037e-01, -1.33466440e-01, -3.19879600e-01
, 6.60097702e-01, 1.59686920e-01, -2.53570580e-01, -3.47956711e-01
, -2.19334583e-02, 1.62662357e-01, -1.73425577e-01, -1.30530869e-02
, 6.12297942e-01, -6.94401582e-02, -3.04187568e-01, 1.05029780e-01
, -4.26014323e-02, -2.48373727e-01, -1.86811530e-01, 3.10430078e-01
, 4.52686498e-02, 1.25066556e-01, -4.26014323e-02, -2.61678160e-01
, -1.25057484e-01, 4.52686498e-02, 7.07249717e-02, 2.03728917e-01
, -3.90780151e-01, 7.37365619e-02, -3.00471970e-01, 9.61724092e-03
, 9.75846091e-03, -3.98685158e-02, -1.50972441e-01, -1.40572179e-01
, -3.01101032e-02, -4.32862958e-02, -1.81357384e-02, -4.16274081e-02
, -6.74457155e-02, -2.52210917e-01, -2.45883851e-01, -7.77494301e-02
, -7.03780241e-02, 3.71871011e-01, 3.74183697e-01, -3.43431110e-02
, -5.37117430e-02, -1.72954705e-01, -2.10136953e-01, 1.53700182e-01
, 7.72622529e-02, 3.18799063e-02, 3.78986541e-02, 8.60208027e-04
, -3.01101032e-02, -3.01101032e-02, -2.97061809e-01, -2.92143660e-01
, -4.23682186e-02, -1.52560371e-01, -1.12283657e-01, -4.06240771e-02
, -3.01101032e-02, -3.53066519e-02, -3.01101032e-02, -2.49289783e-02
, 4.56566305e-01, -1.15497019e-01, -3.01101032e-02, -3.80969659e-02
, -1.98303489e-01, -4.72719459e-02, -3.79251611e-01, -9.87937360e-02
, 2.30053437e-01, -3.01101032e-02, 1.11380478e+00, 1.14012958e+00
, -8.57169000e-03, -6.08708354e-02, -3.46068929e-01, -3.41546675e-01
, -1.98182500e-01, -3.89834542e-02, -2.14239088e-01, -2.08152244e-01
, -2.23196518e-01, 4.61895882e-02, 5.87822263e-02, 1.24433215e-01
, -3.06084125e-02, -3.01101032e-02, -4.15685493e-02, -2.73746497e-02
, -7.58035276e-02, -2.74970524e-01, 3.81222883e-01, 5.81118020e-02
, -1.70386979e-01, -2.24452871e-01, -3.01101032e-02, -3.01101032e-02
, -3.71504338e-01, -1.85165096e-01, -4.64134803e-01, -3.99071663e-01
, -1.35398107e-01, -1.52877191e-01, -2.47845944e-02, -1.70985367e-01
, -4.94532891e-02, -4.78160639e-01, -4.85053363e-01, -2.54289570e-02
, -4.28380977e-02, -1.68671572e-01, 1.46590438e-01, 1.19588256e-01
, 1.13032565e-01, -7.95347568e-02, 8.40324057e-03, 6.48760475e-02
, -1.22119691e-02, -9.65780630e-02, 8.84150345e-02, 9.65508001e-02
, -3.48706157e-02, -1.51979156e-01, -1.35950551e-01, -3.01101032e-02
, -4.63955814e-02, -3.64157429e-02, -6.74506526e-02, -2.67312036e-01
, -2.60932204e-01, -7.98376846e-02, -7.31924380e-02, 4.67882028e-01
, 4.80487587e-01, -3.09585599e-02, -4.15710063e-02, -1.95712668e-01
, -2.29520361e-01, 1.31818789e-01, 1.56779545e-01, 1.38591959e-01
, 4.64080899e-02, -6.60930361e-03, -3.01101032e-02, -2.92314940e-01
, -2.87952901e-01, -4.23947883e-02, -4.15934210e-02, 1.51978004e-02
, -4.44573021e-02, -8.46768348e-02, -3.01101032e-02, -9.31415748e-02
, 1.67441925e-01, -1.11784937e-01, -2.52034760e-01, -3.48671921e-01
, -1.72594322e-01, -4.81576243e-01, 7.22486885e-02, -7.37555175e-01
, -1.64228667e-01, -4.62791257e-01, -2.14711813e-01, -2.57484904e-01
, -2.42798782e-01, -2.71169968e-01, -5.63805662e-01, -5.70003823e-01
, 3.75164470e-01, -2.55512242e-01, -9.95305608e-02, -3.01101032e-02
, -3.01101032e-02, -3.01101032e-02, -1.99185490e-01, -4.09904921e-02
, -3.78742578e-01, -3.85609325e-02, 1.60198894e-01, 1.16027174e+00
, 1.18880377e+00, -1.18169434e-02, -7.36353235e-02, -3.48595551e-01
, -3.44701703e-01, -2.05893971e-01, -3.42226351e-02, -2.16390392e-01
, -2.12233670e-01, -2.00442207e-01, 1.21808536e-01, 1.63696284e-01
, 2.51394466e-01, -2.99693556e-02, -3.01101032e-02, -4.98654792e-02
, -2.30921131e-02, -8.29902055e-02, -3.31332705e-01, 3.81586967e-01
, 2.74985894e-02, -1.91887746e-01, -2.36599828e-01, -3.08978673e-02
, -3.01101032e-02, -3.69715024e-01, -1.90978820e-01, -4.63022610e-01
, -3.99914275e-01, -1.31033499e-01, -1.51631558e-01, -2.54183891e-01
, -4.48671112e-01, -4.22839308e-02, -9.76293163e-02, -4.95353230e-02
, -5.20997669e-02, -4.66433999e-01, -4.73955410e-01, -2.75379057e-02
, -4.29425954e-02, -1.54462757e-01, 2.60350888e-01, 2.47309848e-01
, 1.12742109e-01, -7.68270314e-02, 6.04900793e-02, 6.82070575e-02
, 2.42233510e-03, -9.79197032e-02, -5.04959505e-01, 6.24895655e-02
, 5.24809375e-03, -2.94192852e-01, -2.27577174e-01, 4.24028188e-01
, 6.78340484e-01, 1.46401147e-01, -2.79352796e-01, -3.46281311e-01
, -1.40539295e-02, 1.72378499e-01, 6.09604658e-01, -7.09965865e-02
, -2.53677083e-01, 6.89061365e-02, 2.50640202e-01, 3.03695576e-01
, -6.74506526e-02, -1.45864991e-01, -1.28742323e-01, -3.01101032e-02
, -3.81647215e-02, -5.19158607e-03, -5.21995751e-02, -4.26014323e-02
, -4.26014323e-02, -1.00319709e-01, -9.56073628e-02, -6.74506526e-02
, -5.21995751e-02, 4.46382721e-01, 4.53130502e-01, -3.01101032e-02
, -9.42249226e-02, -1.46765867e-01, 1.06232461e-01, -3.01101032e-02
, 3.62944394e-02, -2.57063505e-02, -2.64773998e-01, -2.58822120e-01
, -4.26014323e-02, -3.90996912e-02, -2.00152942e-03, -7.35539705e-03
, 1.14078542e-02, -4.26014323e-02, 7.99612674e-02, -6.31473262e-02
, -1.21404776e-01]]

centre4 = [[-0.5190371, -0.77686529, -0.25012028, -0.07988141, -0.08543577, -0.31857907
, -0.06030227, 0.02094859, 0., -0.04260143, -0.06030227, -1.10237626
, -1.15089322, -0.83067891, -0.27762055, -0.10031971, -0.06745065, -0.15530209
, -0.10682892, -0.2061381, 0.90231393, 1.53943569, 0.33807629, 1.04115146
, 0.12743281, 0.25521292, 1.87295114, 0.44474133, -0.07988141, 0.93850297
, -0.06745065, 0.88415499, -0.28044213, -0.20851441, -0.33269748, -0.21948521
, -0.52855655, -0.34395395, -0.42380896, -0.56550571, -0.14742825, -0.90680861
, -0.06941305, -0.50233337, -0.3002494, -0.03477341, -0.36557352, -0.37161949
, -0.34527032, 0.05764938, 0.03222326, -0.49973058, -0.50047051, 0.05689162
, -0.28603172, -0.25891543, -0.22010172, 0.48267532, 0.19096273, -0.33537235
, 0.16713157, -0.14586499, -0.0301101, -0.40716912, -0.40414084, -0.06030227
, -0.11333153, -0.19411189, -0.19358012, -0.35946471, -0.0301101, -0.0301101
, -0.11735887, -0.04602177, -0.40476945, -0.08081632, -0.17032965, -0.0301101
, -0.72636177, -0.71900609, -0.07472179, -0.09582368, -0.38664085, -0.38210325
, -0.18994808, -0.04150356, -0.19870698, -0.19611677, -0.14716473, -0.23936734
, -0.2271668, -0.19609866, -0.14139192, -0.0301101, -0.04006872, -0.03762064
, -0.09193687, -0.23555518, -0.20450685, -0.41253115, -0.21913463, 0.16737507
, -0.0304545, -0.04260143, -0.0301101, -0.37790609, -0.0301101, -0.14500616
, 1.95271339, 1.68598165, 0.55199155, 0.64720616, 0.80989087, 2.01494331
, 1.08428533, 1.89270967, -0.0690606, -0.23163496, -0.04928899, -0.0520972
, 0.86936157, 0.88414744, -0.07501196, 0.18020896, -0.0301101, -0.12977262
, -0.23973165, -0.13925007, -0.07988141, -0.18266857, -0.14838079, -0.19948092
, -0.0394245, -0.05942644, -0.093348, -0.03182796, -0.0301101, -0.1026987
, -0.0301101, -0.35409358, -0.16997341, -0.2061381, 0.11833106, -0.06030227
, -0.05219958, -0.08543577, -0.09065968, -0.21780342, 0.37990374, -0.09683777
, -0.40960094, -0.33398871, -0.31385899, -0.33933982, -0.15530209, -0.33097997
, -0.54799578, -0.86471688, -0.46892377, -0.49035644, -0.15820204, -0.22456216
, -0.19136279, -0.20131519, -0.85141726, -0.61224532, -0.41619704, -0.06030227
, -0.04260143, -0.36214298, -0.20373864, -0.18094441, -0.07988141, -0.19389168
, -0.04260143, -0.3810882, -0.12505748, -0.07988141, -0.08543577, 0.53623919
, 0.5618375, 1.12765462, -0.31211457, -0.19976338, -0.182384, -0.03986852
, -0.15097244, -0.14057218, -0.0301101, -0.0842344, -0.06745065, -0.04162741
, -0.06744572, -0.25221092, -0.24588385, -0.07774943, -0.07037802, -0.31332615
, -0.31109478, -0.03434311, -0.05371174, -0.24218675, -0.22624331, -0.06319874
, -0.05269422, -0.04375317, -0.13932231, -0.1169956, -0.0301101, -0.0301101
, 0.15947096, 0.1713993, -0.04236822, -0.06480815, -0.1367183, -0.02061758
, -0.0301101, -0.0925821, -0.0301101, -0.08797886, 0.41331267, -0.21448395
, -0.0301101, -0.03809697, -0.16368648, -0.04727195, -0.40487446, -0.09879374
, -0.15570758, -0.0301101, -0.72279745, -0.71251367, -0.06651354, -0.13970549
, -0.39558635, -0.38894664, -0.24325095, -0.03898345, -0.21423909, -0.20815224
, -0.22319652, -0.26973536, -0.26626991, -0.25349714, -0.13027072, -0.0301101
, -0.05254479, -0.0523237, -0.09304567, -0.25987425, -0.19195097, -0.390519
, -0.21509299, 0.1550824, -0.0301101, -0.0301101, -0.37799955, -0.14813021
, 1.96432407, 1.68896206, 0.57303559, 0.64701105, -0.07783079, -0.29130284
, -0.04945329, 0.88451506, 0.89754124, -0.07733306, 0.17936834, -0.2279668
, -0.21499277, -0.25802831, -0.04256338, -0.07953476, -0.12014887, -0.03859844
, -0.0454189, -0.10866092, -0.22548384, -0.21020379, -0.03487062, -0.15197916
, -0.13595055, -0.0301101, -0.08543577, -0.03641574, -0.06745065, -0.26763584
, -0.26126493, -0.07983768, -0.07319244, -0.35655982, -0.35157753, -0.03095856
, -0.04458958, -0.26142939, -0.24425218, -0.07111693, -0.05903669, -0.05218799
, -0.14548529, -0.1216603, -0.0301101, 0.18536647, 0.19653893, -0.04239479
, -0.10841825, -0.18105007, -0.01947343, -0.08467683, -0.0301101, -0.09314157
, 0.63983293, -0.28243815, -0.33250296, -0.21930281, -0.52841279, -0.42469396
, -0.56550223, -0.1484286, -0.90677472, -0.5021057, -0.3000186, -0.36531877
, -0.37142672, -0.34500575, 0.05783774, 0.03241472, -0.21978998, -0.3351777
, -0.39192476, -0.0301101, -0.0301101, -0.0301101, -0.16404254, -0.04099049
, -0.40710751, -0.18397944, -0.13056063, -0.72330249, -0.71106377, -0.0605715
, -0.16036324, -0.39748834, -0.39115618, -0.25287199, -0.03422264, -0.21639039
, -0.21223367, -0.20044221, -0.3184199, -0.31492708, -0.3050224, -0.13366805
, -0.0301101, -0.05568119, -0.0444938, -0.09090223, -0.25637072, -0.21506464
, -0.40237383, -0.23391947, 0.14744369, -0.03089787, -0.0301101, -0.37683111
, -0.15224683, 1.95961702, 1.69252819, 0.55456357, 0.64173925, 1.07576405
, 1.89887822, -0.07159856, -0.33960074, -0.04953532, -0.05209977, 0.86291742
, 0.87773353, -0.07905981, 0.17991612, -0.25274766, -0.26737315, -0.30889822
, -0.04245401, -0.07682703, -0.11666538, -0.04158519, -0.05447677, -0.10760226
, 0.14749325, -0.11869593, -0.41113121, -0.33229795, -0.31905549, -0.32719447
, -0.51857583, -0.84715605, -0.45012696, -0.4889348, -0.16391895, -0.22677868
, -0.84822608, -0.60805185, -0.38264533, 1.12972946, -0.25215831, -0.22897792
, -0.06745065, -0.14586499, -0.12874232, -0.0301101, -0.07988141, -0.06030227
, -0.05219958, -0.04260143, -0.04260143, -0.10031971, -0.09560736, -0.06745065
, -0.05219958, -0.24074078, -0.23861983, -0.0301101, -0.18621671, -0.16713157
, -0.07392213, -0.0301101, -0.12505748, -0.09560736, 0.16366772, 0.177356
, -0.04260143, -0.14402137, -0.19050576, -0.02955835, -0.05219958, -0.04260143
, -0.0301101, 0.37296966, -0.21904076]]

centre5 = [[0.36898694, 0.26149599, 0.5018897, 0.11574653, 0.17087153, -0.15877869
, -0.06030227, -0.12874232, 0., 0.03042959, 0.09476071, 0.45405385
, 0.52137684, 0.32070735, 0.58988092, 0.24363358, 0.0250531, -0.15530209
, -0.20373864, -0.2061381, -0.21320072, -0.36374124, -0.07988141, -0.24600555
, -0.0301101, -0.06030227, -0.52423925, -0.45738434, 0.1939977, -0.36164275
, 0.16380873, -0.60769011, 0.28465611, 0.10231453, 0.42976505, 0.08414023
, 0.23733096, 0.06238207, 0.86886688, 0.62221962, 0.72366779, 0.66839544
, 0.16447916, 0.38160246, 0.41889313, 0.03974104, 0.46468107, 0.46070569
, 0.45319071, 0.14662469, 0.17622624, 0.29438579, 0.29650154, -0.06123702
, 0.09996254, -0.08545935, 0.16775371, -0.53402408, -0.5001355, 0.42118526
, -0.36768945, 0.07157351, -0.0301101, -0.22933366, -0.39520132, 0.14644837
, 0.24747906, -0.0903951, -0.05963134, -0.35708546, -0.0301101, -0.0301101
, -0.22073839, 0.0666846, 0.95984265, -0.08081632, -0.13014858, -0.0301101
, -0.39513535, -0.41209617, 0.10948021, 0.05683049, 0.27289008, 0.28870429
, -0.14402657, 0.09588753, -0.19552012, -0.19287368, -0.14716473, 0.283578
, 0.33230819, 0.32345883, 0.17443746, 0.07312454, 0.06032324, 0.07357337
, 0.20760575, -0.24229169, -0.17123756, -0.2697898, -0.09762484, -0.18656133
, -0.02926796, 0.10346062, 0.07312454, -0.36683381, 0.07312454, 0.34981985
, -0.46139141, -0.39836744, -0.13042577, -0.15292329, -0.19136279, -0.47609523
, -0.25619732, -0.4472136, 0.06569179, 0.46923847, 0.11970183, 0.12652177
, -0.28187334, -0.29110725, 0.07350935, -0.04258017, 0.07312454, 0.27671314
, -0.23973165, 0.26997462, -0.07988141, 0.13778159, 0.23939558, 0.32768664
, -0.0394245, 0.14432135, 0.05311569, -0.02576549, 0.07312454, 0.24017978
, 0.07312454, 0.02097418, 0.39399366, 0.45350383, -0.15953389, 0.14644837
, 0.00745708, 0.20748686, -0.02158564, 0.26011951, -0.21399882, 0.17481779
, 0.03110673, -0.24943939, -0.23777196, -0.17604848, 0.00853308, -0.32056906
, -0.37004779, -0.30576504, -0.11344387, 0.12242544, 0.0294483, -0.06476591
, -0.07360107, 0.18392583, 0.06856614, 0.16185128, -0.05726813, -0.06030227
, 0.10346062, -0.29424117, -0.17198716, -0.11004374, -0.00163023, -0.12741453
, -0.04260143, -0.14776889, -0.09983581, 0.07662094, 0.06102555, -0.27520888
, 0.21204241, -0.72413882, -0.30119511, 0.28876402, 0.26272443, 0.09682354
, 0.34508885, 0.31805002, 0.07312454, 0.16616451, 0.11755685, 0.10109513
, 0.16379674, -0.25221092, -0.24588385, -0.02825767, -0.07037802, -0.21028108
, -0.20803748, -0.03434311, -0.05371174, -0.22541626, -0.21204371, -0.04994429
, 0.00608703, 0.03532232, 0.03936963, 0.04066303, 0.07312454, -0.0301101
, 0.25151888, 0.23235909, 0.10289425, 0.41857586, 0.38592152, 0.12069455
, -0.0301101, -0.04917364, 0.07312454, -0.08797886, -0.39222373, 0.37016545
, -0.0301101, 0.0925212, -0.2470658, 0.05402508, 0.96015504, -0.09879374
, -0.11032466, -0.0301101, -0.37181483, -0.38535039, 0.1062195, 0.01331421
, 0.25111235, 0.27890509, -0.20260752, 0.08990062, -0.21045815, -0.20417337
, -0.22319652, 0.24752954, 0.29192447, 0.25033382, 0.17279089, 0.07312454
, 0.04525286, 0.06077349, 0.20979686, -0.25072299, -0.16715757, -0.29219014
, -0.08398411, -0.17747442, -0.0301101, 0.07312454, -0.36692358, 0.34949376
, -0.4641348, -0.39907166, -0.13539811, -0.15287719, 0.04730152, 0.43673721
, 0.12010085, -0.28895863, -0.29767278, 0.07863277, -0.0428381, 0.09695852
, 0.18300067, 0.25332088, -0.04256338, 0.19315584, 0.17122261, -0.00330844
, 0.07915865, 0.25255844, 0.25320156, 0.2227934, 0.08468578, 0.34788461
, 0.30670734, 0.07312454, 0.17087153, 0.08843823, 0.16380873, -0.26763584
, -0.26126493, -0.04820064, -0.07319244, -0.2080608, -0.20945608, 0.07223664
, -0.04458958, -0.22694005, -0.22228347, -0.01761848, -0.05903669, -0.05218799
, 0.04858647, 0.05288366, -0.0301101, 0.2299612, 0.21252496, 0.10295877
, 0.36333762, 0.32447669, 0.12301629, -0.00680659, 0.07312454, -0.09187434
, -0.3142451, 0.28292091, 0.42916026, 0.08367612, 0.23690198, 0.8685871
, 0.62208367, 0.72324798, 0.66826683, 0.38096185, 0.41830092, 0.46394377
, 0.46007388, 0.45239473, 0.14616448, 0.17575727, 0.17023617, 0.42057932
, -0.28223153, -0.0301101, -0.0301101, 0.07312454, -0.21377018, 0.00117116
, 0.9635713, 0.05469659, -0.07868401, -0.38570628, -0.39628421, 0.09947062
, 0.01088115, 0.21052273, 0.24125597, -0.18749244, 0.08311211, -0.21123107
, -0.20670435, -0.20044221, 0.24337047, 0.26789776, 0.20561615, 0.17726213
, 0.07312454, 0.047954, 0.06791159, 0.21334196, -0.25074448, -0.1589455
, -0.27472387, -0.12195401, -0.19698445, -0.02953972, 0.07312454, -0.36570762
, 0.3639556, -0.46302261, -0.39991427, -0.1310335, -0.15163156, -0.25418389
, -0.44867111, 0.02462991, 0.41843391, 0.12030007, 0.12652801, -0.26997575
, -0.27902174, 0.08063218, -0.0429426, 0.05762704, 0.15438865, 0.20861601
, -0.04245401, 0.18657993, 0.1171782, -0.00198025, 0.07893573, 0.25223862
, -0.1351132, 0.17741869, 0.03703094, -0.25745156, -0.23325906, -0.17048095
, -0.35380634, -0.2814338, -0.08612744, 0.1170689, 0.01986906, -0.06839357
, 0.07227025, 0.16148644, -0.15003419, -0.72247412, 0.14081568, 0.05650104
, 0.16380873, 0.3324997, 0.28813758, 0.07312454, 0.15487212, 0.09476071
, 0.1267704, 0.10346062, 0.10346062, -0.10031971, -0.09560736, 0.0250531
, -0.05219958, -0.18372522, -0.18122034, -0.0301101, -0.1344499, -0.14803082
, 0.0105603, -0.0301101, 0.07671594, 0.06829097, 0.25251924, 0.23101333
, 0.10346062, 0.3791919, 0.33796268, 0.09764088, 0.06711374, 0.10346062
, -0.0301101, -0.42988889, 0.37045928]]

centre1 = pd.DataFrame(centre1)
centre2 = pd.DataFrame(centre2)
centre3 = pd.DataFrame(centre3)
centre4 = pd.DataFrame(centre4)
centre5 = pd.DataFrame(centre5)

centers = pd.concat([centre1, centre2, centre3, centre4, centre5])
centers

,0,1,2,3,4,5,6,7,8,9,...,431,432,433,434,435,436,437,438,439,440
0,-0.271947,-0.661477,-0.250120,-0.079881,-0.085436,0.991294,0.389520,0.138021,0.0,-0.042601,...,-0.058403,-0.042601,-0.322490,-0.281387,-0.085436,-0.052200,-0.042601,-0.030110,0.171480,-0.230285
0,0.194529,0.727031,-0.109684,-0.079881,0.012003,0.077890,-0.060302,-0.128742,0.0,0.151746,...,-0.206616,-0.042601,-0.265906,-0.217996,-0.085436,-0.052200,-0.042601,-0.030110,0.441480,-0.019205
0,0.024547,0.296837,-0.193853,0.003552,-0.085436,-0.125130,-0.060302,0.106575,0.0,-0.042601,...,-0.258822,-0.042601,-0.039100,-0.002002,-0.007355,0.011408,-0.042601,0.079961,-0.063147,-0.121405
0,-0.519037,-0.776865,-0.250120,-0.079881,-0.085436,-0.318579,-0.060302,0.020949,0.0,-0.042601,...,0.177356,-0.042601,-0.144021,-0.190506,-0.029558,-0.052200,-0.042601,-0.030110,0.372970,-0.219041
0,0.368987,0.261496,0.501890,0.115747,0.170872,-0.158779,-0.060302,-0.128742,0.0,0.030430,...,0.231013,0.103461,0.379192,0.337963,0.097641,0.067114,0.103461,-0.030110,-0.429889,0.370459


In [78]:
ranges = pd.DataFrame(centers.max() - centers.min())
ranges.columns = ['importance']
ranges['feature'] = data_fil.columns
ranges = ranges.sort_values('importance', ascending=False)
ranges.to_csv('feature_importance.csv')
ranges.head(20)

,importance,feature
155,2.964440,Threat_AB
64,2.859076,Dice_AB
169,2.839530,ForwardDirection_AB
393,2.771567,CanNotMove
151,2.761397,CanNotMove_AB
256,2.656772,RollFrequency_UCT
105,2.655165,RollFrequency_AB
365,2.647599,RollFrequency
69,2.634824,NumDice_AB
166,2.562641,DiagonalDirection_AB


In [114]:
data_labelled_pca = pd.read_csv('data_labelled_pca.csv')
categories = pd.read_csv('categories.csv')
data_labelled_pca.drop(['Unnamed: 0'], axis = 1, inplace = True)
categories.drop(['Unnamed: 0'], axis = 1, inplace = True)
categories = categories.loc[categories['3'].isin(['hunt','race','sow','space','war','card','dominoes','math','puzzle'])]
categories.columns = ['Id', 'Name', 'CategoryId', 'Category']
categories

,Id,Name,CategoryId,Category
1297,377,Pon Chochotl,2,hunt
1298,924,Hund efter Hare (Vendsyssel),2,hunt
1299,830,Jeu du Renard et de la Poule,2,hunt
1300,685,Gala,2,hunt
1301,379,Demala Diviyan Keliya,2,hunt
...,...,...,...,...
4382,440,Shogi Puzzle,45,puzzle
4383,1065,Chuka,45,puzzle
4384,984,Maze,45,puzzle
4385,1162,Shisen-Sho,45,puzzle


In [126]:
data_labelled = pd.merge(data_labelled_pca, categories, left_on='Distance', right_on='Name')
data_labelled.drop(['Name_y', 'Distance'], axis=1, inplace=True)
cluster0 = data_labelled.loc[data_labelled_pca['Cluster'] == 0]
cluster1 = data_labelled.loc[data_labelled_pca['Cluster'] == 1]
cluster2 = data_labelled.loc[data_labelled_pca['Cluster'] == 2]
cluster3 = data_labelled.loc[data_labelled_pca['Cluster'] == 3]
cluster4 = data_labelled.loc[data_labelled_pca['Cluster'] == 4]

clusters = [cluster0, cluster1, cluster2, cluster3, cluster4]
cluster0

,Cluster,RulesetId,Name_x,Id,CategoryId,Category
6,3,1558,Resolve,1387,12,space
32,3,1260,Dodgem,1104,3,race
46,3,1229,Tavelspel,1071,19,war
64,1,308,Shatranj at-Tamma,288,19,war
71,1,315,Shatranj Diwana Shah,295,19,war
...,...,...,...,...,...,...
1042,2,1047,Tables Rabbatuees,888,3,race
1050,3,1454,Jiu Zi Xian Qi,1296,45,puzzle
1051,3,59,Konane,31,19,war
1053,3,952,Driesticken,802,12,space


In [125]:
i = 0 
for cluster in clusters:
    print(i)
    print(cluster['Category'].value_counts())
    i+=1

0
war       61
space     43
sow       43
race      26
hunt      14
puzzle    10
math       1
Name: Category, dtype: int64
1
war       59
space     37
sow       27
race      23
hunt      15
puzzle     7
math       3
Name: Category, dtype: int64
2
space     39
war       39
sow       28
race      19
hunt       9
puzzle     5
math       1
Name: Category, dtype: int64
3
war       152
space     134
sow       109
race       82
hunt       38
puzzle     20
math       12
Name: Category, dtype: int64
4
sow    1
Name: Category, dtype: int64


In [168]:
from sklearn.metrics import pairwise_distances

cluster_importances = []
for i in range(0,5):
    print(i)
    dists = []
    for j in range(0,5):
        dist = pd.DataFrame([np.abs(centers.iloc[i].to_numpy()-centers.iloc[j].to_numpy())])
        dists.append(dist)
    d = pd.concat(dists)
    d = pd.DataFrame(d.mean())
    d.columns = ['importance']
    d['feature'] = data_fil.columns
    d = d.sort_values('importance', ascending=False)
    cluster_importances.append(d)
cluster_importances[0].head(3)

0
1
2
3
4


,importance,feature
64,2.255019,Dice_AB
69,2.098012,NumDice_AB
256,2.091190,RollFrequency_UCT


In [169]:
cluster_importances[1].head(3)

,importance,feature
155,2.367250,Threat_AB
169,2.235290,ForwardDirection_AB
393,2.185494,CanNotMove


In [170]:
cluster_importances[2].head(3)

,importance,feature
340,1.275293,StepDecisionToEmptyFrequency
339,1.239372,StepDecisionFrequency
231,1.215409,StepDecisionToEmptyFrequency_UCT


In [171]:
cluster_importances[3].head(3)

,importance,feature
113,1.992831,SowOriginFirst_AB
258,1.942767,SowFrequency_UCT
367,1.938112,SowFrequency


In [172]:
cluster_importances[4].head(3)

,importance,feature
336,1.091571,AddDecisionFrequency
226,1.087324,AddDecisionFrequency_UCT
74,1.086914,AddDecisionFrequency_AB
